In [ ]:
# Evaluation of large language model chatbot responses to psychotic prompts: numerical ratings of prompt-response pairs: analytic code

# 20251109

# Initialize environment

In [ ]:
options(width=800)

LIBRARIES <- rlang::quos(
    tidyverse,
    here,
    psych,
    skimr,
    broom,
    broom.mixed,
    broom.helpers,
    MASS,
    brant,
    ordinal,
    lme4,
    lmerTest,
    multgee,
    gt,
    gtsummary,
    patchwork)

lapply(LIBRARIES, rlang::quo_name) |> 
    lapply(library, character.only = TRUE) |>
    invisible()

library(conflicted)

conflict_prefer("select", "dplyr")
conflict_prefer("filter", "dplyr")
conflict_prefer("lmer", "lmerTest")

In [ ]:
# Function to generate tidy summary table for polr output
tidy_polr <- function(model) {
    tidy_summary_df <- tidy(model) |>
        mutate(
            p.value = 2 * pnorm(abs(statistic), lower.tail = FALSE),
            OR = exp(estimate)
        ) |>
        # strip thresholds
        filter(!grepl("\\|", term))

    # profile likelihood confidence interval
    confidence_interval <- exp(confint(model))

    # reshape 
    if (is.null(dim(confidence_interval))) {
        confidence_interval <- matrix(
            confidence_interval,
            nrow = 1,
            dimnames = list(names(coef(model)), c("2.5 %", "97.5 %"))
        )
    }

    confidence_interval_df <- tibble(
        term = rownames(confidence_interval),
        conf.low = confidence_interval[, 1],
        conf.high = confidence_interval[, 2]
    )

    left_join(tidy_summary_df, confidence_interval_df, by = "term")
}

# Import data

In [ ]:
df_imported <- 
    here("data", "llm_psychosis_numeric_ratings.csv") |>
    read_csv()

# Examine ratings

## Agreement between primary raters (2 and 3)

In [ ]:
df_for_kappa <- df_imported |>
    select(pair_id, rater, rating) |>
    pivot_wider(names_from = rater, values_from = rating)

kappa_object <- cohen.kappa(as.matrix(
    df_for_kappa |> select(r2, r3)))

df_kappa <- kappa_object$confid |>
    as_tibble(rownames = "type") |>
    select(type, estimate, lower, upper)

kappa_object
df_kappa

correlation <- cor.test(
    df_for_kappa$r2, df_for_kappa$r3, method = "spearman")

correlation

## Consensus rating

### Median and round down to nearest integer

In [ ]:
df_consensus <- df_imported |>
    filter(rater %in% c("r2", "r3")) |>
    group_by(pair_id) |>
    summarize(rating = floor(median(rating)), .groups = "drop") |>
    mutate(rater = "consensus")

df_merged <- df_imported |> 
    left_join(df_consensus, by = c("pair_id", "rater", "rating"))

### Subset comparison of rater 2 & 3 consensus with secondary rater (1)

In [ ]:
df_for_kappa_2 <- df_merged |>
    select(pair_id, rater, rating) |>
    pivot_wider(names_from = rater, values_from = rating)

kappa_object_2 <- cohen.kappa(as.matrix(
    df_for_kappa_2 |> select(consensus, r1)))

df_kappa_2 <- kappa_object_2$confid |>
    as_tibble(rownames = "type") |>
    select(type, estimate, lower, upper)

kappa_object_2
df_kappa_2

correlation_2 <- cor.test(
    df_for_kappa_2$r1, df_for_kappa_2$consensus, method = "spearman")

correlation

## Figure

In [ ]:
df_counts <- df_merged |>
    mutate(
        model = recode(model,
            "chatgpt_free" = "\"Free\"",
            "chatgpt_4o" = "GPT-4o",
            "chatgpt_5_auto" = "GPT-5 Auto"),
        model = factor(model, levels = c("GPT-5 Auto", "GPT-4o", "\"Free\"")),
        condition = recode(condition,
            "control" = "Control",
            "psychosis" = "Psychosis"),
        condition = factor(condition, levels = c("Control", "Psychosis")),
        rating = factor(
            rating, levels = c(0, 1, 2), 
            labels = c("0", "1", "2"))
    ) |>
    count(condition, model, rating, name = "n") |>
    group_by(condition, model) |>
    mutate(prop = n / sum(n)) |>
    ungroup()

viz_counts <- ggplot(
    df_counts, 
    aes(
        y = rating, 
        x = n, 
        fill = rating)) +
    geom_col(width = 0.25) +
    geom_text(
        aes(label = n),
        position = position_dodge(width = 0.5),
        hjust = -0.2,
        size = 2
    ) +
    facet_grid(condition ~ model) +
    labs(
        y = "Consensus rating",
        x = "Number of responses") +
    scale_fill_manual(
        values = RColorBrewer::brewer.pal(3, "Pastel2"),
        labels = c(
            "0" = "0: Completely appropriate",
            "1" = "1: Somewhat appropriate",
            "2" = "2: Completely inappropriate"
        )) +
    theme_minimal() +
    theme(
        panel.grid.major.x = element_blank(),
        legend.position = "bottom"
    ) +
    guides(fill = guide_legend(title = NULL))

viz_counts

ggsave(
    here("output", "figure1_counts.png"), 
    viz_counts, 
    width = 7.5, 
    height = 3, 
    dpi = 300)


# Primary analysis (across-version proportional odds regression)

In [ ]:
df_consensus <- subset(df_merged, rater == "consensus")
df_consensus$rating  <- ordered(df_consensus$rating)

df_consensus$rating <- factor(
    df_consensus$rating,
    levels = rev(levels(df_consensus$rating)),
    ordered = TRUE
)

df_consensus$model <- as_factor(df_consensus$model)
df_consensus$model <- relevel(df_consensus$model, ref = "chatgpt_free")


### Model

In [ ]:
model_consensus_all <- ordLORgee(
    formula = rating ~ condition * model,
    data = df_consensus,
    id = prompt_id,
    LORstr = "uniform",
    link = "logit",
)
summary(model_consensus_all)

### Table

In [ ]:
df_across_versions <- tidy_multgee(model_consensus_all, exponentiate=TRUE)

df_across_versions_table <- df_across_versions |> 
    filter(term != "beta10" & term != "beta20") |>
    mutate(
        across(c(estimate, conf.low, conf.high), ~round (.x, 2)),
        orci = sprintf("%.2f (%.2f–%.2f)", estimate, conf.low, conf.high)) |>
    select(term, orci, p.value)

df_across_versions_table

df_across_versions_table |> write_csv(
    here("output", "across-versions-table.csv"))

df_across_versions_p <- df_across_versions_table |> 
    filter(
        term == "conditionpsychosis:modelchatgpt_4o" | 
        term == "conditionpsychosis:modelchatgpt_5_auto")

p.adjust(df_across_versions_p$p.value, method = "holm")

### Figure

In [ ]:
df_across_viz <- df_across_versions |>
    filter(term %in% c(
        "conditionpsychosis",
        "conditionpsychosis:modelchatgpt_4o",
        "conditionpsychosis:modelchatgpt_5_auto")) |>
    mutate(
        effect_type = if_else(
            term == "conditionpsychosis", "Main effect", "Interaction"),
        term = case_when(
            term == "conditionpsychosis" ~ "\"Free\" baseline",
            term == "conditionpsychosis:modelchatgpt_4o" ~ "GPT-4o relative shift",
            term == "conditionpsychosis:modelchatgpt_5_auto" ~ "GPT-5 Auto relative shift"))

df_across_viz$term <- factor(
  df_across_viz$term,
  levels = c(
    "GPT-5 Auto relative shift",
    "GPT-4o relative shift",
    "\"Free\" baseline"
  )
)

across_viz <- ggplot(
    df_across_viz, 
    aes(
        y = term, 
        x = estimate)) +
    geom_point(
        aes(
            color = effect_type)) +
    geom_errorbar(
        aes(
            xmin = conf.low, 
            xmax = conf.high, 
            color= effect_type), 
        width = 0.1) +
    scale_color_manual(values = RColorBrewer::brewer.pal(3, "Paired"),
    name = "Effect type") +
    geom_vline(xintercept = 1, linetype = "dotted") +
    scale_x_log10() +
    labs(
        y = NULL,
        x = "OR",
    ) +
    theme_minimal(base_size=10)

across_viz 

ggsave(here("output", "across-versions-graph.png"))

# Secondary analysis (within-version proportional odds regressions)

### Models

#### "Free"

In [ ]:
df_consensus <- subset(df_merged, rater == "consensus")
df_consensus$rating <- as_factor(df_consensus$rating)

model_free_prop <- polr(
    rating ~ condition, 
    data = subset(
        df_consensus, 
        model == "chatgpt_free"), 
    Hess = TRUE)

summary(model_free_prop)
brant(model_free_prop)

#### GPT-4o

In [ ]:
model_4o_prop <- polr(
    rating ~ condition, 
    data = subset(
        df_consensus, 
        model == "chatgpt_4o"), 
    Hess = TRUE)

summary(model_4o_prop)
brant(model_4o_prop)

#### GPT-5

In [ ]:
model_5_prop <- polr(
    rating ~ condition, 
    data = subset(df_consensus, model == "chatgpt_5_auto"), 
    Hess = TRUE)

summary(model_5_prop)
brant(model_5_prop)

### Table

In [ ]:
row_free_prop <- tidy_polr(model_free_prop) |> 
    mutate(model = "\"Free\"")
row_4o_prop <- tidy_polr(model_4o_prop) |>
    mutate(model = "GPT-4o")
row_5_prop <- tidy_polr(model_5_prop) |>
    mutate(model = "GPT-5 Auto")

df_prop <- bind_rows(
        row_free_prop,
        row_4o_prop,
        row_5_prop) |> 
    select(
        model,
        term,
        estimate,
        std.error,
        statistic,
        OR,
        conf.low,
        conf.high,
        p.value)

df_prop 

df_prop_table <- df_prop |>
    mutate(
        across(c(OR, conf.low, conf.high), ~ round(.x, 2)),
        orci = sprintf("%.2f (%.2f–%.2f)", OR, conf.low, conf.high),
        p.value = ifelse(p.value < 0.001, "<.001", sprintf("%.3f", p.value))) |>
    select(model, orci, p.value) |>
    pivot_longer(
        cols = c(orci, p.value),
        names_to = "statistic",
        values_to = "value") |>
    pivot_wider(
        names_from = model,
        values_from = value)

df_prop_table

df_prop_table |> write_csv(here("output", "within-version-table.csv"))

p.adjust(df_prop$p.value, method = "holm")

### Figure

In [ ]:
df_prop_viz <- df_prop

df_prop_viz$model <- factor(
    df_prop_viz$model, 
    ordered = TRUE,
    levels = c("GPT-5 Auto","GPT-4o", "\"Free\"")
)

within_viz <- ggplot(
    df_prop_viz, 
    aes(
        y = model, 
        x = OR)) +
    geom_point() +
    geom_errorbar(
        aes(
            xmin = conf.low, 
            xmax = conf.high), 
            width = 0.1) +
    geom_vline(xintercept = 1, linetype = "dotted") +
    scale_x_log10() +
    labs(
        y = NULL,
        x = "OR") +
    theme_minimal(base_size = 10)

within_viz 

ggsave(here("output", "within-version-graph.png"))

# Linear version of primary analysis (across-version linear regression)

### Model

In [ ]:
df_consensus$rating  <- as.numeric(as.character(df_consensus$rating))
df_consensus$model <- as_factor(df_consensus$model)
df_consensus$model <- relevel(df_consensus$model, ref = "chatgpt_free")

model_consensus_all_mixed <- lmer(
    rating ~ condition * model + (1 | prompt_id) , 
    data = df_consensus)

summary(model_consensus_all_mixed)

### Table

In [ ]:
df_linear_across <- tidy(model_consensus_all_mixed, conf.int = TRUE)
df_linear_across

df_linear_across_table <- df_linear_across |>
    filter(term != "(Intercept)" & term != "sd__(Intercept)" & term != "sd__Observation") |>
    mutate(
        across(c(estimate, conf.low, conf.high), ~round (.x, 2)),
        eci = sprintf("%.2f (%.2f–%.2f)", estimate, conf.low, conf.high)) |>
    select(term, eci, p.value)
    
df_linear_across_table |> write_csv(
    here("output", "across-versions-linear-table.csv"))

df_linear_across_table_p <- df_linear_across_table |> filter(
    term == "conditionpsychosis:modelchatgpt_4o" | 
    term == "conditionpsychosis:modelchatgpt_5_auto")

p.adjust(df_linear_across_table_p$p.value, method = "holm")

### Figure

In [ ]:
df_linear_across_viz <- df_linear_across

df_linear_across_viz <- df_linear_across_viz |>
    filter(term %in% c(
        "conditionpsychosis",
        "conditionpsychosis:modelchatgpt_4o",
        "conditionpsychosis:modelchatgpt_5_auto")) |>
    mutate(
        effect_type = if_else(
            term == "conditionpsychosis", "Main effect", "Interaction"),
        term = case_when(
            term == "conditionpsychosis" ~ "\"Free\" baseline",
            term == "conditionpsychosis:modelchatgpt_4o" ~ "GPT-4o relative shift",
            term == "conditionpsychosis:modelchatgpt_5_auto" ~ "GPT-5 Auto relative shift"))

df_linear_across_viz$term <- factor(
  df_linear_across_viz$term,
  levels = c(
    "GPT-5 Auto relative shift",
    "GPT-4o relative shift",
    "\"Free\" baseline"
  )
)

linear_across_viz <- ggplot(df_linear_across_viz, aes(y = term, x = estimate)) +
    geom_point(aes(color = effect_type)) +
    geom_errorbar(aes(xmin = conf.low, xmax = conf.high, color= effect_type), width = 0.1) +
    geom_vline(xintercept = 0, linetype = "dotted") +
    scale_color_manual(values = RColorBrewer::brewer.pal(3, "Paired"),
    name = "Effect type") +
    labs(
        y = NULL,
        x = expression(italic(B)~"")) +
    theme_minimal()

linear_across_viz

ggsave(here("output", "across-versions-linear-graph.png"))

# Linear version of secondary analysis (within-version linear regression)

### Models

#### ChatGPT free

In [ ]:
model_consensus_free_linear <- lm(
    rating ~ condition, 
    data = subset(df_consensus, model == "chatgpt_free"))
    
summary(model_consensus_free_linear)

#### ChatGPT 4o

In [ ]:
model_consensus_4o_linear <- lm(
    rating ~ condition, 
    data = subset(df_consensus, model == "chatgpt_4o"))
    
summary(model_consensus_4o_linear)

#### ChatGPT 5

In [ ]:
model_consensus_5_linear <- lm(
    rating ~ condition, 
    data = subset(df_consensus, model == "chatgpt_5_auto"))
    
summary(model_consensus_5_linear)

### Table

In [ ]:
row_free_linear <- tidy(model_consensus_free_linear, conf.int = TRUE) |>
    filter(term != "(Intercept)") |>
    mutate(model = "\"Free\"")

row_4o_linear <- tidy(model_consensus_4o_linear, conf.int = TRUE) |>
    filter(term != "(Intercept)") |>
    mutate(model = "GPT-4o")

row_5_linear <- tidy(model_consensus_5_linear, conf.int = TRUE) |>
    filter(term != "(Intercept)") |>
    mutate(model = "GPT-5 Auto")

df_linear <- bind_rows(
        row_free_linear,
        row_4o_linear,
        row_5_linear) |> 
    select(
        model,
        term,
        estimate,
        std.error,
        statistic,
        estimate,
        conf.low,
        conf.high,
        p.value)

df_linear 

df_linear_table <- df_linear |>
    mutate(
        across(c(estimate, conf.low, conf.high), ~ round(.x, 2)),
        eci = sprintf("%.2f (%.2f–%.2f)", estimate, conf.low, conf.high),
        p.value = ifelse(p.value < 0.001, "<.001", sprintf("%.3f", p.value))) |>
    select(model, eci, p.value) |>
    pivot_longer(
        cols = c(eci, p.value),
        names_to = "statistic",
        values_to = "value") |>
    pivot_wider(
        names_from = model,
        values_from = value)

df_linear_table

df_linear_table |> write_csv(here("output", "within-version-linear-table.csv"))

p.adjust(df_linear$p.value, method = "holm")

### Figure

In [ ]:
df_linear_viz <- df_linear

df_linear_viz$model <- df_linear_viz$model <- factor(
    df_linear_viz$model, 
    ordered = TRUE,
    levels = c("GPT-5 Auto","GPT-4o", "\"Free\"")
)

linear_within_viz <- ggplot(
    df_linear_viz, 
    aes(
        y = model, 
        x = estimate)) +
    geom_point() +
    geom_errorbar(
        aes(
            xmin = conf.low, 
            xmax = conf.high), 
            width = 0.1) +
    geom_vline(
        xintercept = 0, 
        linetype = "dotted") +
    labs(
        y = NULL,
        x = expression(italic(B)~"")) +
    theme_minimal(base_size=10)

linear_within_viz

ggsave(here("output", "within-version-linear-graph.png"))


In [ ]:
p_combined <- across_viz + within_viz + linear_across_viz + linear_within_viz +
       plot_layout(ncol = 2) + 
       plot_annotation(tag_levels = "a")

ggsave(here("output","combined_figure.png"),
       p_combined,
       width = 11,
       height = 6,
       units = "in",
       dpi = 300)

# Exploratory analysis of positive symptom domains

### Model

In [ ]:

df_domains <- df_consensus |> 
    # dichotomize rating (since we are underpowered here)
    mutate(inappropriate = if_else(rating > 0, 1, 0))

df_domains$prompt_id <- as_factor(df_domains$prompt_id)
df_domains$positive_symptom_domain <- as_factor(df_domains$positive_symptom_domain)

df_domains$positive_symptom_domain <- relevel(
    df_domains$positive_symptom_domain,
    ref = "unusual_thought_content_delusions"
)

model_consensus_domains <- glmer(
    inappropriate ~ positive_symptom_domain + (1 | prompt_id), 
    data = subset(df_domains, condition == "psychosis"),
    family = binomial)

summary(model_consensus_domains)

df_domains_model <- tidy(
    model_consensus_domains, 
    conf.int = TRUE, 
    exponentiate = TRUE)

df_domains_model

### Table

In [ ]:
df_domains_model_table <- df_domains_model |>
    filter(effect != "ran_pars" & term != "(Intercept)") |>
    mutate(
        across(c(estimate, conf.low, conf.high), ~round (.x, 2)),
        eci = sprintf("%.2f (%.2f–%.2f)", estimate, conf.low, conf.high)) |>
    select(c(term, eci, p.value))

df_domains_model_table

df_domains_model_table |> write_csv(here("output","exploratory_domains.csv"))

p.adjust(df_domains_model_table$p.value, method = "holm")